In [145]:
# Import Packages
import numpy
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score
import time

In [146]:
# Read in data to pandas dataframes
train = pd.read_csv('training.txt', header = None)
train = train.rename(columns={0: "text", 1: "good"})
X_train = train['text']
y_train = train['good']
test = pd.read_csv('testing.txt', header = None)
test = test.rename(columns={0: "text", 1: "good"})
X_test = test['text']
y_test = test['good']

In [160]:
# Run Naive bayes algorithm
start_nb = time.time()
cv = CountVectorizer(stop_words='english')
train_cv = cv.fit_transform(X_train)
test_cv = cv.transform(X_test)
mnb = MultinomialNB()
mnb.fit(train_cv, y_train)
predictions = mnb.predict(test_cv)
end_nb = time.time()

In [161]:
# Report results
print('Accuracy:', accuracy_score(y_test, predictions))
print('Precision:', precision_score(y_test, predictions))
print('Recall:', recall_score(y_test, predictions))
print('Time:', (end_nb - start_nb), 'seconds')

Accuracy: 0.8460076045627376
Precision: 0.8505747126436781
Recall: 0.8363269424823411
Time: 5.81065821647644 seconds
